<a href="https://colab.research.google.com/github/george-kan/fastai/blob/main/Mnist_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook

     |████████████████████████████████| 727kB 18.0MB/s 
     |████████████████████████████████| 51kB 9.6MB/s 
     |████████████████████████████████| 1.2MB 57.5MB/s 
     |████████████████████████████████| 194kB 59.7MB/s 
     |████████████████████████████████| 61kB 11.4MB/s 


In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
(path/'training').ls()

(#10) [Path('/root/.fastai/data/mnist_png/training/8'),Path('/root/.fastai/data/mnist_png/training/3'),Path('/root/.fastai/data/mnist_png/training/4'),Path('/root/.fastai/data/mnist_png/training/0'),Path('/root/.fastai/data/mnist_png/training/6'),Path('/root/.fastai/data/mnist_png/training/9'),Path('/root/.fastai/data/mnist_png/training/5'),Path('/root/.fastai/data/mnist_png/training/7'),Path('/root/.fastai/data/mnist_png/training/1'),Path('/root/.fastai/data/mnist_png/training/2')]

In [5]:
folder = 'training/3'

(path/folder).ls()

(#6131) [Path('/root/.fastai/data/mnist_png/training/3/45599.png'),Path('/root/.fastai/data/mnist_png/training/3/46500.png'),Path('/root/.fastai/data/mnist_png/training/3/41404.png'),Path('/root/.fastai/data/mnist_png/training/3/10993.png'),Path('/root/.fastai/data/mnist_png/training/3/983.png'),Path('/root/.fastai/data/mnist_png/training/3/27840.png'),Path('/root/.fastai/data/mnist_png/training/3/17333.png'),Path('/root/.fastai/data/mnist_png/training/3/29097.png'),Path('/root/.fastai/data/mnist_png/training/3/40205.png'),Path('/root/.fastai/data/mnist_png/training/3/39761.png')...]

In [6]:
(path/'testing').ls()

(#10) [Path('/root/.fastai/data/mnist_png/testing/8'),Path('/root/.fastai/data/mnist_png/testing/3'),Path('/root/.fastai/data/mnist_png/testing/4'),Path('/root/.fastai/data/mnist_png/testing/0'),Path('/root/.fastai/data/mnist_png/testing/6'),Path('/root/.fastai/data/mnist_png/testing/9'),Path('/root/.fastai/data/mnist_png/testing/5'),Path('/root/.fastai/data/mnist_png/testing/7'),Path('/root/.fastai/data/mnist_png/testing/1'),Path('/root/.fastai/data/mnist_png/testing/2')]

In [7]:
train_x = []
train_y = []
test_x = []
test_y = []
for digit in range(10):
    folder = 'training/' + str(digit)
    new_inp = [tensor(Image.open(x)) for x in (path/folder).ls()]
    train_x += new_inp
    train_y += [digit]*len(new_inp)
    test_folder = 'testing/' + str(digit)
    test_inp = [tensor(Image.open(x)) for x in (path/test_folder).ls()]
    test_x += test_inp
    test_y += [digit]*len(test_inp)
    

In [8]:
train_x_stacked = torch.stack(train_x).view(-1, 28*28).float()/255
train_y_stacked = tensor(train_y).view(-1, 1)
train_ds = list(zip(train_x_stacked, train_y_stacked))
train_dl = DataLoader(train_ds, batch_size= 256, shuffle=True)

test_x_stacked = torch.stack(test_x).view(-1, 28*28).float()/255
test_y_stacked = tensor(test_y).view(-1, 1)
test_ds = list(zip(test_x_stacked, test_y_stacked))
test_dl = DataLoader(test_ds, batch_size=256)

dls = DataLoaders(train_dl, test_dl)

train_x_stacked.shape, train_y_stacked.shape, test_x_stacked.shape, test_y_stacked.shape

(torch.Size([60000, 784]),
 torch.Size([60000, 1]),
 torch.Size([10000, 784]),
 torch.Size([10000, 1]))

In [9]:
train_x[2000]

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  12,  92, 214, 255, 254, 176, 138,  33,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 143, 253, 253, 253, 253, 253, 253, 2

In [10]:
df = pd.DataFrame(torch.trunc(tensor(train_x[2000]).float()))
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,92.000000,214.000000,255.000000,254.000000,176.000000,138.000000,33.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,143.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,236.000000,58.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,88.000000,250.000000,221.000000,93.000000,10.000000,27.000000,192.000000,251.000000,253.000000,240.000000,61.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,54.000000,246.000000,199.000000,28.000000,0.000000,0.000000,0.000000,0.000000,76.000000,230.000000,253.000000,208.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,150.000000,215.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,79.000000,251.000000,253.000000,142.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
def batch_accuracy(x, y):
    preds = torch.argmax(x, 1)
    correct = preds == y
    return correct.float().mean()

simple_net = nn.Sequential(
            nn.Linear(28*28, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 10)
)
loss = CrossEntropyLossFlat()
learn = Learner(dls, simple_net, opt_func = SGD, loss_func = loss, metrics = batch_accuracy)
learn.fit(40, 0.3)


epoch,train_loss,valid_loss,batch_accuracy,time
0,0.342673,0.273259,0.844673,00:01
1,0.211816,0.181899,0.866217,00:01
2,0.167797,0.272501,0.841799,00:01
3,0.144669,0.131778,0.883945,00:01
4,0.122090,0.131966,0.882782,00:01
5,0.103729,0.173585,0.869960,00:01
6,0.090275,0.111314,0.887486,00:01
7,0.082080,0.118197,0.885019,00:01
8,0.081032,0.115382,0.887778,00:01
9,0.074782,0.095189,0.893688,00:01


In [12]:
x,y = dls.one_batch()
learn.get_preds(dl = [(x,y)])

(tensor([[3.1124e-09, 6.3867e-09, 2.8526e-08,  ..., 2.0649e-05, 5.0099e-11, 8.0734e-06],
         [3.6380e-10, 6.4586e-14, 8.7704e-14,  ..., 7.6045e-18, 1.4278e-10, 1.6293e-09],
         [1.6650e-11, 9.7846e-09, 2.0936e-07,  ..., 1.0000e+00, 1.2848e-09, 2.0325e-10],
         ...,
         [7.4114e-09, 1.9172e-10, 2.2734e-10,  ..., 3.9139e-15, 9.4640e-09, 1.9306e-07],
         [9.9187e-01, 1.9093e-14, 8.1285e-03,  ..., 3.0295e-09, 1.1574e-08, 1.6320e-06],
         [3.8186e-16, 1.0000e+00, 7.8526e-10,  ..., 3.7289e-08, 1.3166e-07, 1.0766e-11]]),
 tensor([[4],
         [6],
         [7],
         [5],
         [3],
         [3],
         [4],
         [3],
         [3],
         [2],
         [0],
         [5],
         [3],
         [4],
         [0],
         [7],
         [1],
         [2],
         [4],
         [3],
         [6],
         [3],
         [1],
         [0],
         [9],
         [1],
         [2],
         [8],
         [3],
         [4],
         [3],
         [3],
  

In [13]:
??Learner

In [128]:
def init_params(size1, size2): return torch.randn(size1, size2).requires_grad_()

def my_linear(weights, bias, x): return x@weights + bias

def my_relu(x): return torch.maximum(tensor(0), x)

def my_softmax(x): 
    # Normalize the input to avoid numerical problems
    x = x - torch.max(x, dim = 1).values.view(-1, 1)
    return torch.exp(x) / torch.exp(x).sum(dim = 1).unsqueeze(1)

def my_loss(x, y): 
    x = my_softmax(x)
    y = y.view(-1, 1)
    selection = torch.gather(x, 1, y)
    #print(selection)
    return (1-selection).mean()

def my_model(x):
    lay1 = my_linear(w1, b1, x)
    nonlin = my_relu(lay1)
    preds = my_linear(w2, b2, nonlin)
    return(preds)

def update_grads(preds, y, params):
    loss = my_loss(preds, y)
    #print(loss)
    loss.backward()
    for p in params:
        my_grad = p.grad
        p.data -= my_grad*lr
        print(p.mean())
        print(my_grad.mean())
        p.grad.zero_()

def eval_batch():
    preds = my_model(test_x_stacked)
    positions = torch.max(preds, dim = 1).indices
    return (positions == test_y_stacked).float().mean()

In [158]:
inputs, classes = next(iter(train_dl))  

In [179]:
preds = my_model(inputs)
update_grads(preds, classes, params)
my_loss(my_model(inputs), classes)

tensor(0.0124, grad_fn=<MeanBackward0>)
tensor(2.6444e-34)
tensor(1.1252, grad_fn=<MeanBackward0>)
tensor(8.1041e-32)
tensor(-0.0665, grad_fn=<MeanBackward0>)
tensor(0.)
tensor(-0.2970, grad_fn=<MeanBackward0>)
tensor(-3.6734e-40)


tensor(0.8984, grad_fn=<MeanBackward0>)

In [182]:
my_softmax(my_model(inputs))[:,0]
classes

tensor([[9],
        [2],
        [0],
        [1],
        [3],
        [6],
        [4],
        [3],
        [2],
        [7],
        [5],
        [1],
        [7],
        [2],
        [7],
        [2],
        [3],
        [2],
        [3],
        [7],
        [9],
        [6],
        [4],
        [9],
        [3],
        [9],
        [3],
        [6],
        [6],
        [5],
        [0],
        [5],
        [1],
        [3],
        [8],
        [4],
        [3],
        [0],
        [3],
        [5],
        [8],
        [0],
        [6],
        [7],
        [6],
        [2],
        [2],
        [2],
        [1],
        [2],
        [8],
        [4],
        [7],
        [8],
        [4],
        [4],
        [9],
        [1],
        [5],
        [5],
        [1],
        [9],
        [8],
        [4],
        [6],
        [2],
        [0],
        [2],
        [3],
        [7],
        [8],
        [4],
        [2],
        [2],
        [4],
        [0],
        [2],

In [87]:
blu.sum(dim = 1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000], grad_fn=<SumBackward1>)

In [83]:
train_y_stacked[:5]

tensor([[0],
        [0],
        [0],
        [0],
        [0]])

In [89]:
-torch.log(torch.gather(blu, 1, train_y_stacked[:5]))

tensor([[    inf],
        [66.3725],
        [    inf],
        [26.4826],
        [71.3144]], grad_fn=<NegBackward>)

In [153]:
w1 = init_params(28*28, 20)
b1 = init_params(1, 1)
w2 = init_params(20, 10)
b2 = init_params(1, 1)
lr = 10
params = w1, b1, w2, b2

tuple

In [72]:
def my_model(x):
    lay1 = my_linear(w1, b1, x)
    nonlin = my_relu(lay1)
    preds = my_linear(w2, b2, nonlin)
    return(preds)

def update_grads(preds, y, params):
    loss = my_loss(preds, y)
    #print(loss)
    loss.backward()
    for p in params:
        grad = p.grad
        p.data -= grad*lr
        print(grad.mean())
        p.grad = None

def eval_batch():
    preds = my_model(test_x_stacked)
    positions = torch.max(preds, dim = 1).indices
    return (positions == test_y_stacked).float().mean()


In [18]:
eval_batch()

tensor(0.0999)

In [79]:
update_grads(my_model(train_x_stacked[:13]), train_y_stacked[:13], params)

tensor(3.0244e-15)
tensor(3.9658e-13)
tensor(2.0905e-21)
tensor(1.1285e-21)


In [41]:
preds = my_model(train_x_stacked[0])
update_grads(preds, train_y_stacked[0], params)

tensor([[0.4991]], requires_grad=True)


tensor([[0.4976]], requires_grad=True)

In [67]:
for i in range(20):
  for x,y in train_dl:
      preds = my_model(x)
      update_grads(preds, y, params)
      #print('break')
  print(w1.mean(), w2.mean())

tensor(-0.4492, grad_fn=<MeanBackward0>) tensor(-0.0200, grad_fn=<MeanBackward0>)
tensor(-0.4267, grad_fn=<MeanBackward0>) tensor(-0.0200, grad_fn=<MeanBackward0>)
tensor(-0.4544, grad_fn=<MeanBackward0>) tensor(-0.0200, grad_fn=<MeanBackward0>)
tensor(-0.4840, grad_fn=<MeanBackward0>) tensor(-0.0200, grad_fn=<MeanBackward0>)
tensor(-0.4772, grad_fn=<MeanBackward0>) tensor(-0.0200, grad_fn=<MeanBackward0>)
tensor(-0.4982, grad_fn=<MeanBackward0>) tensor(-0.0200, grad_fn=<MeanBackward0>)
tensor(-0.4744, grad_fn=<MeanBackward0>) tensor(-0.0200, grad_fn=<MeanBackward0>)
tensor(-0.4882, grad_fn=<MeanBackward0>) tensor(-0.0200, grad_fn=<MeanBackward0>)
tensor(-0.4963, grad_fn=<MeanBackward0>) tensor(-0.0200, grad_fn=<MeanBackward0>)
tensor(-0.5085, grad_fn=<MeanBackward0>) tensor(-0.0200, grad_fn=<MeanBackward0>)
tensor(-0.5138, grad_fn=<MeanBackward0>) tensor(-0.0200, grad_fn=<MeanBackward0>)
tensor(-0.5310, grad_fn=<MeanBackward0>) tensor(-0.0200, grad_fn=<MeanBackward0>)
tensor(-0.5440, 

tensor(0.1010)

In [19]:
train_x_stacked[:2].shape, train_y_stacked[:2].shape

(torch.Size([2, 784]), torch.Size([2, 1]))

In [20]:
lvl1 = my_linear(w1, b1, train_x_stacked[:2])
nonlin = my_relu(lvl1)
pred = my_linear(w2, b2, nonlin)
loss = my_centloss(pred, train_y_stacked[:2])
loss.backward()
w1.data -= w1.grad*lr
b1.data -= b1.grad*lr
w2.data -= w2.grad*lr
b2.data -= b2.grad*lr
w1.grad = None
b1.grad = None
w2.grad = None
b2.grad = None

In [21]:
w1.grad.data

AttributeError: ignored

In [ ]:
pred.shape
torch.exp(pred)
torch.max(pred, dim = 1).values.view(-1, 1)

In [ ]:
pred - torch.max(pred, dim = 1).values.view(-1, 1)

In [ ]:
pred

In [ ]:
test_tensor = tensor([1,-2,3], [-2,7,4]).float()
test_tensor.shape

In [ ]:
(torch.exp(test_tensor) / torch.exp(test_tensor).sum(dim = 1).unsqueeze(1)).sum(1)

In [ ]:
my_softmax(tensor([1,1,3], [-2,7,1]).float())
(tensor(1) - tensor([[0.1065], [0.0025]])).mean()

In [ ]:
my_centloss(tensor([1,1,3], [-2,7,1]).float(), tensor([1,2]))

In [ ]:
test_tensor = torch.tensor([[1,-2,3], [-2,7,4], [1,1,1]]).float()
select_tensor = torch.tensor([1,2, 0], dtype=torch.int64).view(-1,1) # number of dimension should match with the test tensor.
final_tensor = torch.gather(test_tensor, 1, select_tensor)
final_tensor

In [ ]:
torch.gather(test_tensor, 0, select_tensor)